In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [2]:
import bigframes.pandas as bpd

Set the display option to use anywidget

In [3]:
bpd.options.display.repr_mode = "anywidget"

Display the dataframe in anywidget mode

In [4]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

Computation deferred. Computation will process 171.4 MB


Display Series in anywidget mode

In [5]:
test_series = df["year"]
print(test_series)

Computation deferred. Computation will process 171.4 MB
